# Question 1

#### Create a query that provides the following information for all items ordered.

In [ ]:
CREATE TABLE OrderDetailCostsTable AS
SELECT [OrderDetails].OrderID,[OrderDetails].Quantity as OrderQuantity,[Products].Price as UnitPrice,[OrderDetails].Quantity*[Products].Price as QuantityCost FROM [OrderDetails]
JOIN [Products]
  ON [Products].ProductID == [OrderDetails].ProductID
    
CREATE TABLE OrderCostsTable AS
SELECT OrderID,SUM(OrderQuantity) as TotalQuantity,SUM(QuantityCost) as TotalCost FROM [OrderDetailCostsTable]
GROUP BY OrderID

SELECT OrderDetailID, ProductName, Quantity as ProductQuantity, Price as UnitPrice, price*quantity AS TotalProductPrice, CategoryName, [OrderDetails].OrderID, OrderDate, [Customers].CustomerID, Country as CustomerCountry, TotalQuantity	TotalCost, [Employees].LastName FROM [OrderDetails]
JOIN [Products] 
  ON [OrderDetails].ProductID == [Products].ProductID
JOIN [Categories]
  ON [Categories].CategoryID == [Products].CategoryID
JOIN [Orders]
  ON [Orders].OrderID == [OrderDetails].OrderID
JOIN [Customers]
  ON [Customers].CustomerID == [Orders].CustomerID
JOIN [Employees]
  ON [Employees].EmployeeID == [Orders].EmployeeID
JOIN [OrderCostsTable]
  ON [OrderCostsTable].OrderID == [OrderDetails].OrderID

# Question 2

#### 1: Which employee has sold the most products?

In [ ]:
SELECT [Orders].EmployeeID,[Employees].FirstName,[Employees].LastName,SUM(TotalQuantity) as ProductsSold FROM [Orders]
JOIN [OrderCostsTable]
  ON [OrderCostsTable].OrderID == [Orders].OrderID
JOIN [Employees]
  ON [Employees].EmployeeID == [Orders].EmployeeID
GROUP BY [Orders].EmployeeID
ORDER BY ProductsSold DESC

Answer: Margaret Peacock sold 3232 items

#### 2: Which product has generated the most revenue for the business?

In [ ]:
SELECT [Products].ProductName,SUM([OrderDetails].Quantity*[Products].Price) as Revenue FROM [OrderDetails]
JOIN [Products]
  ON [Products].ProductID == [OrderDetails].ProductID
GROUP BY [Products].ProductName
ORDER BY Revenue DESC

Answer: Côte de Blaye with 62976.5 (dollars presumably)

#### 3: Which category has produced the least revenue for the business?

In [ ]:
SELECT [Categories].CategoryName,SUM([OrderDetails].Quantity*[Products].Price) as Revenue FROM [OrderDetails]
JOIN [Products]
  ON [Products].ProductID == [OrderDetails].ProductID
JOIN [Categories]
  ON [Categories].CategoryID == [Products].CategoryID
GROUP BY [Categories].CategoryName
ORDER BY Revenue

Answer: Grains/Cereals with 22327.75 (dollars)

#### 4: How many customers have ordered once, more than once or never?

In [ ]:
CREATE TABLE CustomerOrderCountsTable AS
SELECT [Customers].CustomerID, [Customers].CustomerName, COUNT(*) as CustomerOrders FROM [Customers]
JOIN [Orders]
  ON [Customers].CustomerID == [Orders].CustomerID
GROUP BY [Customers].CustomerID
ORDER BY CustomerOrders DESC

SELECT COUNT(*) FROM [CustomerOrderCountsTable]
WHERE CustomerOrders > 1

SELECT COUNT(*) FROM [CustomerOrderCountsTable]
WHERE CustomerOrders == 1

Answer:
Customers ordered greater than once: 52
Customers ordered exactly once: 22
Customers never ordered: There are 91 customers in the customers table, therefore 91 - 52 - 22 = 17 customers have never ordered

#### 5: Which shipper has shipped the most products? (FYI, the text in the HW pdf is incorrect and the updated query + text was agreed upon with Zach)

In [ ]:
SELECT [Shippers].ShipperName, SUM([OrderDetails].Quantity) as TotalShipped FROM [OrderDetails]
JOIN [Orders]
  ON [OrderDetails].OrderID == [Orders].OrderID
JOIN [Shippers]
  ON [Shippers].ShipperID == [Orders].ShipperID
GROUP BY [Shippers].ShipperID
ORDER BY TotalShipped DESC

Answer: United Package has shipped the most with 4802 units

#### 6: What percentage of orders by $ are Dairy products as a percentage of all order value by employee?

In [ ]:
CREATE TABLE EmployeeCategoriesTable AS
SELECT [Orders].EmployeeID, CategoryName, SUM(price*quantity) AS Revenue FROM [OrderDetails]
JOIN [Products] 
  ON [OrderDetails].ProductID == [Products].ProductID
JOIN [Categories]
  ON [Categories].CategoryID == [Products].CategoryID
JOIN [Orders]
  ON [Orders].OrderID == [OrderDetails].OrderID
GROUP BY [Orders].EmployeeID, CategoryName


CREATE TABLE EmployeeDairyRevenueTable AS
SELECT * FROM [EmployeeCategoryRevenueTable]
WHERE CategoryName == "Dairy Products"



SELECT [Employees].FirstName, [Employees].LastName, ROUND(100*[EmployeeDairyRevenueTable].Revenue/SUM([EmployeeCategoryRevenueTable].Revenue),1) as PercentSalesOfDairy FROM [EmployeeDairyRevenueTable]
JOIN [EmployeeCategoryRevenueTable]
  ON [EmployeeCategoryRevenueTable].EmployeeID == [EmployeeDairyRevenueTable].EmployeeID
JOIN [Employees]
  ON [Employees].EmployeeID == [EmployeeDairyRevenueTable].EmployeeID
GROUP BY [EmployeeDairyRevenueTable].EmployeeID
ORDER BY PercentSalesOfDairy DESC

#### 7: Which employee has shipped the most orders over $1500 by count to a country that has a vowel as its fourth letter?

In [ ]:
CREATE TABLE EmployeeCountryOrders AS
SELECT [Orders].OrderID, [Orders].CustomerID, [Customers].Country, [OrderCalcTable].TotalCost, [Orders].EmployeeID, [Employees].FirstName, [Employees].LastName FROM [Orders]
JOIN [Employees]
  ON [Employees].EmployeeID == [Orders].EmployeeID
JOIN [OrderCalcTable]
  ON [Orders].OrderID == [OrderCalcTable].OrderID
JOIN [Customers]
  ON [Customers].CustomerID == [Orders].CustomerID
    
    
SELECT EmployeeID, FirstName, LastName, COUNT(*) as EmployeeOrders FROM [EmployeeCountryOrders]
WHERE TotalCost >= 1500 and substr(Country, 4, 1) in ('a', 'e', 'i', 'o', 'u')
GROUP BY EmployeeID
ORDER BY EmployeeOrders DESC

#### Answer: 
EmployeeID	FirstName	LastName	EmployeeOrders
3	Janet	Leverling	3
7	Robert	King	2
4	Margaret	Peacock	2
9	Anne	Dodsworth	1
6	Michael	Suyama	1
2	Andrew	Fuller	1
1	Nancy	Davolio	1

#### 8: For customers with more than one order, what was the amount of time elapsed between their last order and the one prior to that?

In [ ]:
CREATE TABLE CustomerLastDateTable AS
SELECT [Customers].CustomerID, MAX([Orders].OrderDate)  FROM [Customers]
JOIN [Orders]
  ON [Customers].CustomerID == [Orders].CustomerID
JOIN [CustomerOrderCountsTable]
  ON [Customers].CustomerID == [CustomerOrderCountsTable].CustomerID
WHERE [CustomerOrderCountsTable].CustomerOrders > 1
GROUP BY [Customers].CustomerID


CREATE TABLE CustomerDateDeltaTable AS
SELECT [Customers].CustomerID, [Orders].OrderDate, [CustomerLastDateTable].LastDate, JulianDay([Orders].OrderDate) -  JulianDay([CustomerLastDateTable].LastDate) as DateDelta  FROM [Customers]
JOIN [Orders]
  ON [Customers].CustomerID == [Orders].CustomerID
JOIN [CustomerOrderCountsTable]
  ON [Customers].CustomerID == [CustomerOrderCountsTable].CustomerID
JOIN [CustomerLastDateTable]
  ON [Customers].CustomerID == [CustomerLastDateTable].CustomerID
WHERE [CustomerOrderCountsTable].CustomerOrders > 1 and DateDelta < 0


SELECT CustomerID,-1*MAX(DateDelta) as TimeBetweenLastAndPenultimatePurchase FROM [CustomerDateDeltaTable]
GROUP BY CustomerID

CustomerID	TimeBetweenLastAndPenultimatePurchase (Days)
4	31
5	124
7	75
9	27
10	20
14	145
17	27
19	36
20	12
21	27
24	55
25	58
28	29
29	60
34	2
35	163
36	21
37	48
38	2
39	2
41	10
44	73
46	23
48	13
49	116
51	15
55	108
58	41
59	34
60	38
61	41
62	34
63	56
65	57
66	15
67	15
68	192
69	32
71	42
72	10
73	79
75	45
76	63
80	20
81	54
83	33
84	105
85	27
86	11
87	20
88	190
89	93

#### Bonus: How many orders has the Batman sold to customers whose names end in a consonant?

In [ ]:
SELECT OrderID,	CustomerID,	[Employees].EmployeeID, [Employees].FirstName as EmployeeFirstName, [Employees].LastName as EmployeeLastName  FROM [Orders]
JOIN [Employees]
  ON [Employees].EmployeeID == [Orders].EmployeeID
WHERE EmployeeFirstName in ('Adam') and EmployeeLastName in ('West')

Answer: 0

# Question 3: Please generate a report that shows how many items from each category each employee has sold, showing the employee name, product name, and quantity sold

### I believe the question should read as "showing the employee name, category name, and quantity sold. That said, I ran three versions of this query in case I misunderstood the text

#### By Product Name (Exclusive)

In [ ]:
SELECT [Employees].FirstName as EmployeeFirstName, [Employees].LastName as EmployeeLastName, [Products].ProductName, Sum([OrderDetails].Quantity) as QuantitySold  FROM [OrderDetails]
JOIN [Orders]
  ON [Orders].OrderID == [OrderDetails].OrderID
JOIN [Employees]
  ON [Employees].EmployeeID == [Orders].EmployeeID
JOIN [Products]
  ON [Products].ProductID == [OrderDetails].ProductID
GROUP BY [Employees].EmployeeID, [Products].ProductID

#### By Category Name (Inclusive)

In [ ]:
SELECT [Employees].FirstName as EmployeeFirstName, [Employees].LastName as EmployeeLastName, [Categories].CategoryName, Sum([OrderDetails].Quantity) as QuantitySold  FROM [OrderDetails]
JOIN [Orders]
  ON [Orders].OrderID == [OrderDetails].OrderID
JOIN [Employees]
  ON [Employees].EmployeeID == [Orders].EmployeeID
JOIN [Products]
  ON [Products].ProductID == [OrderDetails].ProductID
JOIN [Categories]
  ON [Categories].CategoryID == [Products].CategoryID
GROUP BY [Employees].EmployeeID, [Products].CategoryID

#### By Both (A combination in case this is what was requested)

In [ ]:
SELECT [Employees].FirstName as EmployeeFirstName, [Employees].LastName as EmployeeLastName, [Categories].CategoryName, [Products].ProductName, Sum([OrderDetails].Quantity) as QuantitySold  FROM [OrderDetails]
JOIN [Orders]
  ON [Orders].OrderID == [OrderDetails].OrderID
JOIN [Employees]
  ON [Employees].EmployeeID == [Orders].EmployeeID
JOIN [Products]
  ON [Products].ProductID == [OrderDetails].ProductID
JOIN [Categories]
  ON [Categories].CategoryID == [Products].CategoryID
GROUP BY [Employees].EmployeeID, [Categories].CategoryID, [Products].ProductID

# Question 4: As you are probably aware, in Email syntax, the local part (before @) can incarnate in several forms that all pertain to the same recipient. E.g., candidate@creditninja.com, candidate+2019@creditninja.com and Can.di.Date@creditninja.com are all equivalent and pointing to the same email address. Using SQL in a flavor of your choosing, write a statement that recognizes any and all combinations of the above as the same.

#### I did not put this code into a functional form, but presumably that would be the place for it. For each possible value of the variable `email`, the `trueemail` variable is `candidate@creditninja.com` as requested.

In [ ]:
DECLARE @email VARCHAR(64);
DECLARE @atidx INT;
DECLARE @plsidx INT;
DECLARE @domain VARCHAR(64);
DECLARE @username VARCHAR(64);
DECLARE @trueemail VARCHAR(64);

#SELECT @email = 'candidate@creditninja.com';
#SELECT @email = 'candidate+2019@creditninja.com';
#SELECT @email = 'Can.Di.Date@creditninja.com';


SELECT @atidx = CHARINDEX('@', @email, 0);

SELECT @username = SUBSTRING(@email, 0, @atidx);
SELECT @domain   = SUBSTRING(@email, @atidx, len(@email));

SELECT @plsidx   = CHARINDEX('+', @username, 0);
SELECT @plsidx   = IIF(@plsidx > 0, @plsidx, len(@username)+1);
SELECT @username = SUBSTRING(@username, 0, @plsidx);
SELECT @username = LOWER(@username);
SELECT @username = REPLACE(@username, '.', '');

SELECT @trueemail = CONCAT(@username,@domain)
SELECt @trueemail